<a href="https://colab.research.google.com/github/angelamyungjoosong/study_data_analytics/blob/main/codes/NLP/healthapp_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://heytech.tistory.com/401

In [1]:
!pip install python-mecab-ko pyLDAvis

In [2]:
import numpy as np

In [3]:
#파일 읽기
import pandas as pd
df_raw = pd.read_excel('./dataset_raw.xlsx')
df_raw[:2]

,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5


#전처리

## 결측치 처리

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


In [5]:
df_raw_dropna = df_raw.dropna() #결측치 제거
df_raw_dropna.isnull().sum() #결측치 개수 확인

app       0
review    0
rating    0
dtype: int64

## 자연어 전처리

#### 특정 (상업용) 앱 리스트에서 제거

In [6]:
remove_app_list = pd.read_excel('./remove_app_list.xlsx')
remove_app_list[:5]

,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


In [7]:
remove_app_list.info() #106 업체명 -> 삭제 대상

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes


In [8]:
from pandas._libs.tslibs.parsing import parse_datetime_string_with_reso
for remove_app in remove_app_list['app']:
  try:
    df_removed_app = df_raw_dropna[df_raw_dropna['app'] != remove_app]
  except:
    pass
  #df_raw_dropna에서 remove_app_list에 있는 걸 제외시키기

In [31]:
df_removed_app = df_removed_app[500:9000]

### 형태소 적용과 불용어 처리

In [32]:
#불용어 사진 불러오기
df_stopwords = pd.read_excel('./stopword_list.xlsx')
df_stopwords[:5]


,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [33]:
from mecab import MeCab
mecab = MeCab()
stopwords = df_stopwords['stopword'].values #df_stopwords['stopword']는 series이기 때문에 values만 뽑음
#토크나이징 함수 정의(한 자 이상만 적용, 불용어 삭제) #전부 다 noun으로 바꾸기
def tokenizer(raw,  stopword=stopwords):
  result_list = list()
  for word in mecab.nouns(raw):
    #한 자 이상만 적용, 불용어 삭제
        if len(word) > 1 and word not in stopword :
            result_list.append(word) #리스트로 넣어 join해서 문자열로 바꾸어 리턴
  #일부 경우 (토큰 갯수 제한)  #보통 토큰이 3개 이상이어야 유의미하다고 봄
  #if len(result_list) <=  3 or len(result_list) < 12 :
  #     result_list
  return ' '.join(result_list) #return str -> dataframe으로 들어가도록

In [34]:
#형태 분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂')

'운동'

In [35]:
mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂')

['곳', '운동', '수']

In [36]:
df_removed_app['tokenized_review'] = df_removed_app['review'].apply(tokenizer)
#review가 하나씩 raw로 들어간다
df_removed_app
#tokenized_review는 날려버린다

<ipython-input-36-4613aa8c4253>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_removed_app['tokenized_review'] = df_removed_app['review'].apply(tokenizer)


,app,review,rating,tokenized_review,replaced_review
1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,,
1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,
1002,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무좋아요,5,,
1003,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,짱,5,,
1004,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,말이 필요없다. 운동 앱중에선 단연 이 어플이 최고!,5,필요 운동 최고,
...,...,...,...,...,...
8995,30일만에 뱃살 빼기 - 평평한 배 만들기,따라하기도싶고 잼있게 할수있네요..추천입니다,5,추천,
8996,30일만에 뱃살 빼기 - 평평한 배 만들기,운동하기 좋고 영상까지 나와 효과적입니다.,5,운동 영상,
8997,30일만에 뱃살 빼기 - 평평한 배 만들기,참 좋은앱!ㅎㅎㅎ,5,,
8998,30일만에 뱃살 빼기 - 평평한 배 만들기,쉽고 재미있고 별로 힘들지 않아서 좋아요! 한번 깔아보세요! 추천합니다!! 살 빼고...,5,추천,


#### 유사어 대치  
- 주제어를 뽑아내기 위해 특정 단어를 같은 부류의 단어로 치환시킴 (ex.G3, G4 -> 스마트폰 / 푸쉬업 -> 푸시업 )

In [37]:
df_replace_list = pd.read_excel('./replace_list.xlsx')
df_replace_list[:3]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰


In [38]:
def replace_word(tokenized_review):
  replaced_review = str()
  for i in range(len(df_replace_list['before_replacement'])):
    try:
      # 치환할 단어가 있는 경우에만 데이터 치환 수행
      if df_replace_list['before_replacement'][i] in tokenized_review:
        replaced_review = tokenized_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
      else:
        replaced_review = tokenized_review  # 치환할 단어가 없는 경우, 원래 단어로 리턴
    except Exception as e:
      print(f"Error 발생 / 에러명: {e}")
  return replaced_review

In [39]:
replace_word ('S헬스 한글 LG V10 업데이트 완벽')

'S헬스 한글 LG V10 업데이트 완벽'

In [40]:
replace_word('한글 업데이트 완벽')

'한글 업데이트 완벽'

In [41]:
df_removed_app['replaced_review'] = df_removed_app['tokenized_review'].apply(replace_word)
df_removed_app[:2]

<ipython-input-41-97e3c3bb3ffe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_removed_app['replaced_review'] = df_removed_app['tokenized_review'].apply(replace_word)


,app,review,rating,tokenized_review,replaced_review
1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,,
1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고


In [42]:
df_removed_app.to_csv('./preprocess_reviews.csv')

### 부정과 긍정 댓글 분리

In [43]:
condition_positive = df_removed_app['rating'] >= 3  #긍정
condition_negative = df_removed_app['rating'] < 3  #부정

In [44]:
df_review_positive = df_removed_app[condition_positive]
df_review_negative= df_removed_app[condition_negative]

In [45]:
df_review_positive[:3]

,app,review,rating,tokenized_review,replaced_review
1000,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,👍,5,,
1001,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,힘들지만 살이 빠지는것 같아요. 땀이 많이나요 진짜 말을 할수가 없어...,5,최고,최고
1002,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무좋아요,5,,


In [46]:
df_review_negative[:3]

,app,review,rating,tokenized_review,replaced_review
1072,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,너무 힘들다,1,,
1167,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,훌륭,1,,
1192,매일 운동 - 운동 & 피트니스 트레이너 - 빠르고 효율적인 운동,좋아요,1,,


In [52]:
df_review_negative.info()
df_review_negative.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1072 to 8916
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               413 non-null    object
 1   review            413 non-null    object
 2   rating            413 non-null    int64 
 3   tokenized_review  413 non-null    object
 4   replaced_review   413 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.4+ KB


<ipython-input-52-b00c97406bef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_negative.dropna(inplace=True)


In [53]:
df_review_positive.info()
df_review_positive.dropna(inplace=True)
# df_review_positive = df_review_positive.dropna

<class 'pandas.core.frame.DataFrame'>
Index: 7587 entries, 1000 to 8999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               7587 non-null   object
 1   review            7587 non-null   object
 2   rating            7587 non-null   int64 
 3   tokenized_review  7587 non-null   object
 4   replaced_review   7587 non-null   object
dtypes: int64(1), object(4)
memory usage: 355.6+ KB


<ipython-input-53-883a33e96700>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_positive.dropna(inplace=True)


In [54]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 1072 to 8916
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               413 non-null    object
 1   review            413 non-null    object
 2   rating            413 non-null    int64 
 3   tokenized_review  413 non-null    object
 4   replaced_review   413 non-null    object
dtypes: int64(1), object(4)
memory usage: 19.4+ KB


### 긍정, 부정으로 전처리한 것을 csv 파일로 저장


## LDA 분석 (다른 파일에)

In [49]:
df_review_positive.to_csv('./review_positive.csv')
df_review_negative.to_csv('./review_negative.csv')